In [ ]:
import pandas as pd
import numpy as np
from dateutil.parser import parse 
import matplotlib
import matplotlib.pyplot as plt
from numpy.random import randn
from pandas import Series, DataFrame
from pylab import rcParams
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
###
from pandas import read_csv
from pandas import datetime

# Importing the datasets

In [ ]:
df = pd.read_csv(r"C:\Users\arrah\Desktop\Trendminer_Intern\data_one_day.csv", parse_dates =["timestamp"])

#df_large = pd.read_csv(r"C:\Users\arrah\Desktop\Trendminer_Intern\CoffeeML-07-20_08-08.csv",index_col=0,squeeze=True,parse_dates =["timestamp"])

count_row = df.shape[0]  # gives number of row count
count_column = df.shape[1] # gives number of column count

print('Number of rows: {}'.format(count_row))
print('Number of columns: {}'.format(count_column))

df.head()

# Specifying the range of time needed

In [ ]:
#data_0 = df1.drop(df1['2020-07-16 00:00:00':'2020-07-16 07:50:00'].index)
df['timestamp'] = pd.to_datetime(df['timestamp'])
startDate=['2020-09-02 06:00:00']
endDate=['2020-09-02 19:00:00']
#data_0[data_0.timestamp<endDate & startDate]
#df[df['timestamp']>= startDate & df ['timestamp']<= endDate]
result = []
for i in  range(0, len(startDate)) :
    start = startDate[i]
    end = endDate[i]
    result.append((df['timestamp'] > start) & (df['timestamp'] <= end)) 

resultDf = []
for item in result:
    if len(item)>0:
        resultDf.append(df.loc[item])

df_select = pd.concat(resultDf)
df_select

In [ ]:
# Check for missing data
df_select.isnull().sum() # gives number of null value by column

# setting the timestamp variable as index

In [ ]:
df_index=df_select.set_index('timestamp')
df_index

In [ ]:
df_index.index

In [ ]:
# Resampling the datasets in 100ml
df_resample=df_index.resample('100ms',level=0).mean()
df_resample

In [ ]:
#Splitting the dataset into dependent variable and independent variables, so as to apply interpollation
X_i, y_i = df_resample.drop(['Keypad.keyPressed'], axis=1), df_resample['Keypad.keyPressed']
X_i

In [ ]:
#interpollation of the independent variables
X_inter = X_i.interpolate(method='linear', axis=0).ffill().bfill()
X_inter

In [ ]:
# concantenate the interpollated dataset to y_i
df_con = pd.concat([X_inter, y_i], axis=1)
df_con

In [ ]:
# Boxplot
import seaborn as sb
sb.boxplot(x='Keypad.keyPressed', y='AnalogNoiseSensor.noiseTotal', data=df_con, palette='hls')

In [ ]:
# correlation plot
plt.figure(figsize=(7,7))
p=sns.heatmap(X_inter.corr(), annot=True,cmap='RdYlGn',square=True, fmt='.2f')

In [ ]:
# Extracting our variables of interest
df_extr = df_con[['AnalogNoiseSensor.noiseTotal','Keypad.keyPressed']]
df_extr

In [ ]:
# resetting the index column as a variable
df_reset= df_extr.reset_index()
df_reset

In [ ]:
# delete all rows with column 'Keypad.keyPressed having  values 8 to 9 
df_reset1= df_reset[~df_reset['Keypad.keyPressed'].isin([8,9])] 
df_reset1

In [ ]:
#g = df_reset1.to_csv(r"C:\Users\arrah\Desktop\Trendminer_Intern\data_one_day_transformed.csv", index = False)

In [ ]:
import plotly.express as px
fig = px.line(df_reset1, x='timestamp',y = 'AnalogNoiseSensor.noiseTotal',title= 'timestamp vs AnalogNoiseSensor.noiseTotal with slider')

fig.update_xaxes(
     rangeslider_visible=True,
     rangeselector=dict(
       
     )
)
fig.show()

In [ ]:
df_reset2=df_reset1[35000:40000]
df_reset2

In [ ]:
# import plotly.express as px
# fig = px.line(db, x='timestamp',y = 'AnalogNoiseSensor.noiseTotal',title= 'timestamp vs AnalogNoiseSensor.noiseTotal with slider')

# fig.update_xaxes(
#      rangeslider_visible=True,
#      rangeselector=dict(
#         buttons=list([
#             dict(count=1, label="1 day", step="day", stepmode="backward"),
#             dict(count=2, label="4 day", step="day", stepmode="backward"),
#             dict(count=3, label="7 day", step="day", stepmode="backward"),
#             dict(step="all")
#         ])
#      )
# )
# fig.show()

In [ ]:
# df_analog =df_reset[['AnalogNoiseSensor.noiseTotal']]
# df_analog

# Sliding window

In [ ]:
global_len=5000
global_k=60
# Applying only for the variable analognoise.total

def generate_sample(k, sample_list):
    result = []
    for i in range(len(sample_list)):
        if i < len(sample_list) - k:
            result.append(sample_list[i : i+k])           
    temp = np.array(result)
    final_result = np.reshape(temp, (1, len(sample_list)-k, k))
    return final_result
    


data_matrix = df_reset2["AnalogNoiseSensor.noiseTotal"].values

sampled_matrix_of_k = generate_sample(60, data_matrix)

new_analog= pd.DataFrame(sampled_matrix_of_k[0])
new_analog
#sampled_matrix_of_k[0][0]

In [ ]:
# Applying only for the variable keypad pressed
import math

def sliding_window_of_values(key_pad_frame, k):
    keypad_matrix = key_pad_frame.values
    
    r = []
    for i in range(len(keypad_matrix) - k):
        print(keypad_matrix[i : i+k])
        for y in range(len(keypad_matrix[i : i+k])):
            if not math.isnan(keypad_matrix[i : i+k][y]):
            
                r.append(keypad_matrix[i : i+k][y])
                break
            elif y == len(keypad_matrix[i : i+k]) - 1:
                r.append(keypad_matrix[i : i+k][y])
        
    return r
                
test_dataframe = df_reset2["Keypad.keyPressed"]

keypadd = sliding_window_of_values(test_dataframe, 60)

In [ ]:
keypadd

In [ ]:
print(len(keypadd))

In [ ]:
df_timestamp =df_reset[["timestamp"]][:global_len-global_k]
df_timestamp

# Concantenating the frames

In [ ]:
new_analog['keypad'] = keypadd

In [ ]:
new_analog

In [ ]:
merged_frame = pd.concat([df_timestamp, new_analog], axis=1)
merged_frame.head()

In [ ]:
# counts the number of occurences greater than target_value in a list of integer
def count_occurence_in_row(row, target_value):
    count = 0
    for i in range(len(row)):
        if row[i] > target_value:
            count += 1
    return count

In [ ]:
# returns true if the count in a row is greater or equal than the cutt_off_pt
def is_valid_row(row, target_value, cut_off_pt):
    if count_occurence_in_row(row, target_value) >= cut_off_pt:
        return True
    else:
        return False

In [ ]:
# creating a  new dataframe satisfying the conditions of the target avlue and cutt_off_pt 

def create_data_frame_with_valid_row(sample_data_frame, cutt_off_pt, target_value):
    analog_df_values = sample_data_frame.drop(columns=['timestamp', 'keypad']).values
    invalid_indexes = []
    for index in range(len(analog_df_values)):
        if not is_valid_row(analog_df_values[index], target_value, cutt_off_pt):
            invalid_indexes.append(index)
    return sample_data_frame.drop(sample_data_frame.index[invalid_indexes])

In [ ]:
result_frame = create_data_frame_with_valid_row(merged_frame, cutt_off_pt=45, target_value=15300)

result_frame

In [ ]:
#g = result_frame.to_csv(r"C:\Users\arrah\Desktop\Trendminer_Intern\exploratory.csv", index = True)

In [ ]:
#using the index as a list to do the filtering
result_frame.index.tolist()

In [ ]:
# the function uses the list of index and takes the largest from the first column(zero) 
#of consecutive sequence base on the difference we chose

def drop_consecutive_row(input_df, delta ):
    index_list = input_df.index.tolist()
    index_to_keep = []
    start = 0
    for i in range(len(index_list)-1):
        if index_list[i+1] - index_list[i] > delta:
            index_to_keep.append(input_df[0][start:i].idxmax(axis=0))
            start = i+1
            
    return input_df.loc[index_to_keep]

In [ ]:
selecte_frame = drop_consecutive_row(result_frame, 100)
selecte_frame

In [ ]:
#g = df_one_day.to_csv(r"C:\Users\arrah\Desktop\Trendminer_Intern\final_dataframe\Data_one_day_70_15200_45_60_final.csv", index = True)

In [ ]:
# df_part1 = pd.read_csv(r"C:\Users\arrah\Desktop\Trendminer_Intern\final_dataframe\new\After_df_part1_70_15200_45_70.csv")
# df_part1 = df_part1.drop(['Unnamed: 0'], axis=1)
# df_part1

In [ ]:
# df_part2 = pd.read_csv(r"C:\Users\arrah\Desktop\Trendminer_Intern\final_dataframe\new\After_df_part2_70_15200_45_70.csv")
# df_part2 = df_part2.drop(['Unnamed: 0'], axis=1)
# df_part2

In [ ]:
# df_part3 = pd.read_csv(r"C:\Users\arrah\Desktop\Trendminer_Intern\final_dataframe\new\After_df_part3_70_15200_45_70.csv")
# df_part3 = df_part3.drop(['Unnamed: 0'], axis=1)
# df_part3

In [ ]:
# df_part11 = pd.read_csv(r"C:\Users\arrah\Desktop\Trendminer_Intern\final_dataframe\new\After_df_part11_70_15200_45_70.csv")
# df_part11 = df_part11.drop(['Unnamed: 0'], axis=1)
# df_part11

In [ ]:
# df_one_day = pd.read_csv(r"C:\Users\arrah\Desktop\Trendminer_Intern\final_dataframe\Data_one_day_70_15200_45_60_final.csv")
# df_one_day.head()

In [ ]:
# dropping columns not needed
df_one_day1 = df_one_day.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)
df_one_day1.head()

In [ ]:
# df_append = pd.concat([df_part1,df_part2,df_part3,df_part11, df_one_day1])
# df_append

In [ ]:
df_append = pd.read_csv(r"C:\Users\arrah\Desktop\Trendminer_Intern\final_dataframe\new\df_join1.csv")
df_append

In [ ]:
# splitting the dataset to apply clustering on the predictors 
X_indep , Y_dep= df_append1.drop(['timestamp', 'keypad'], axis=1),df_append1['keypad']
X_indep

# Clustering Models

setting up for cluster Analysis

In [ ]:
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import scale
import sklearn.metrics as sm
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.figure(figsize=(7,4))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
#scaler = StandardScaler()

# Optionnal
X_scale= pd.DataFrame(scaler.fit_transform(X_indep), columns=X_indep.columns)
X_scale

# Building and running your model

In [ ]:
clusters = 5
kmeans = KMeans(n_clusters = clusters, random_state=123) 
y_assign=kmeans.fit(X_scale) 
y_assign
#print(y_assign.labels_)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=123)
kmeans.fit(X_scale)
y_kmeans = kmeans.predict(X_scale)
y_kmeans

In [ ]:
## count of each class
frame = pd.DataFrame(X_scale)
frame['cluster'] = y_kmeans
frame['cluster'].value_counts()

In [ ]:
df_append['cluster'] = y_kmeans
df_append.head()

# Hierachical Clustering

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
%matplotlib inline 
from sklearn.decomposition import PCA 
from sklearn.cluster import AgglomerativeClustering 
from sklearn.preprocessing import StandardScaler, normalize 
from sklearn.metrics import silhouette_score 
import scipy.cluster.hierarchy as shc 

In [ ]:
sts = X_scale.iloc[:,:].values
sts[:,0]

# CLUSTERING Create Linkage Matrix

In [ ]:
Z = linkage(sts, 'ward')

Plot Dendrogram of Clusters

In [ ]:
plt.figure(figsize = (25, 10))
plt.title('Hierachical Cluster Dendrogram ')
plt.axhline(y=5, color='r', linestyle='--')
#plt.ylabel('distance')
dendrogram(
    Z,
    labels =X_scale.index,
    leaf_rotation = 0.,
    leaf_font_size = 18.,
)
plt.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering
cluster_h = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')  
hl = cluster_h.fit_predict(sts)
hl

In [ ]:
## count of each class for hierachical clustering
frame = pd.DataFrame(X_scale)
frame['cluster'] = hl
frame['cluster'].value_counts()

In [ ]:
df_append['cluster'] = hl
df_append

In [ ]:
plt.figure(figsize=(10, 7))
plt.scatter(sts[:,0], sts[:,0], c=cluster_h.labels_, cmap='rainbow')

# PCA ANALYSIS

In [ ]:
from sklearn.decomposition import PCA   
pca = PCA(0.8, svd_solver='full') 
pca.fit(X_scale) 
  
pca_data = pd.DataFrame(pca.transform(X_scale)) 
  
pca_data

In [ ]:
# df_pca = pd.concat([pca_data,y], axis=1) 
# df_pca

In [ ]:
pca.explained_variance_ratio_

In [ ]:
pca.singular_values_

In [ ]:
pca.explained_variance_

In [ ]:
# random oversampling to balance the class distribution
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
X_o, y_o= make_classification(n_samples=61, n_features=19, n_informative=3,
                            n_redundant=0, n_repeated=0, n_classes=5,
                            n_clusters_per_class=1,
                            weights=[0.48, 0.28, 0.13, 0.07, 0.04],
                            class_sep=0.8, random_state=123)

In [ ]:
df_pca_o = pd.DataFrame(X_o)
df_pca_o.columns = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18']
df_pca_o['keypad']= y
df_pca_o.head()

In [ ]:
df_pca_o['keypad'].astype(str).hist()

#apply the Random over-sampling approach.

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=123)

In [ ]:
X_train_oversample, y_train_oversample = ros.fit_resample(X_o, y_o)

In [ ]:
df_pca_o = pd.DataFrame(y_train_oversample)
df_pca_o[0].astype(str).hist()

In [ ]:
df_bal= pd.Series(data=y_resampled.flatten())
df_bal.value_counts()

In [ ]:
# Handling Imbalance using SMOTE
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.datasets import make_classification
from matplotlib import pyplot
from collections import Counter
from numpy import where
#pip install imblearn
#pip install imbalanced-learn
smote = SMOTE(sampling_strategy = 'minority')

In [ ]:
# define dataset
X_sm, y_sm = make_classification(n_samples=61, n_features=19, n_redundant=0, n_classes=5, n_informative=3,
n_clusters_per_class=1, weights=[0.48, 0.28, 0.13, 0.07, 0.04], flip_y=0, random_state=123)

In [ ]:
df_pca_sm = pd.DataFrame(X_sm)
df_pca_sm.columns = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18']
df_pca_sm['keypad']= y
df_pca_sm.head()

In [ ]:
df_pca_sm['keypad'].astype(str).hist()

In [ ]:
# summarize class distribution
counter = Counter(y_sm)
print(counter)

In [ ]:
# transform the dataset
sm = SMOTE(sampling_strategy='auto', k_neighbors=1, random_state=123)
X_smote, y_smote = sm.fit_resample(X_sm, y_sm)
# summarize the new class distribution
counter = Counter(y_smote)
print(counter)

In [ ]:
df_pca_sm = pd.DataFrame(y_smote)
df_pca_sm[0].astype(str).hist()

In [ ]:
X_train_smote, y_train_smote = smote.fit_resample(X_smote, y_smote)

In [ ]:
# # example of evaluating a model with random oversampling and undersampling
# from numpy import mean
# from sklearn.datasets import make_classification
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import RepeatedStratifiedKFold
# from sklearn.tree import DecisionTreeClassifier
# from imblearn.pipeline import Pipeline
# from imblearn.over_sampling import RandomOverSampler

In [ ]:
# # define pipeline
# over = RandomOverSampler(sampling_strategy=0.1)
# steps = [('o', over), ('m', DecisionTreeClassifier())]
# pipeline = Pipeline(steps=steps)
# # evaluate pipeline
# cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=123)
# scores = cross_val_score(pipeline, X, y, scoring='f1_micro', cv=cv, n_jobs=-1)
# score = mean(scores)
# print('F1 Score: %.3f' % score)

In [ ]:
 # drop NaN values from df_append on the keypad to apply classification modelling on the label data
df_append1 = df_append.dropna()
df_append1

In [ ]:
# k=pd.DataFrame(data=y_smote.flatten())
# k

# Normalize

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X, y = pca_data , df_append1['keypad']

scaler = MinMaxScaler()
#scaler = StandardScaler()

# Optionnal
X = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns)
X

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# X, y = df_append1.drop(['timestamp','keypad'], axis=1) , df_append1['keypad']

# scaler = MinMaxScaler()
# #scaler = StandardScaler()

# # Optionnal
# X = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns)
# X

# SPLIT TRAIN\TEST

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train_oversample, y_train_oversample , test_size=0.2, random_state=123)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=1,solver='lbfgs',max_iter=100)

clf.fit(X_train, y_train)

preds = clf.predict(X_test)
#print('Test set predictions: {}'.format(preds))
#print('\n')

#print('Train set accuracy: {:.2f}'.format(accuracy_score(y_train, clf.predict(X_train))))
#print('Test set accuracy: {:.2f}'.format(accuracy_score(y_test, clf.predict(X_test))))
print('Train set accuracy: {:.2f}'.format(clf.score(X_train, y_train)))
print('Test set accuracy: {:.2f}'.format(clf.score(X_test, y_test)))
print('\n')

confusion = confusion_matrix(y_test, preds)

print('Confusion matrix:\n', confusion)

# plt.bar(range(X_train.shape[1]), clf.coef_[0], align = 'center', alpha = 0.5)
# plt.xticks(range(X_train.shape[1]), X_train.columns, rotation=90)
# xlims = plt.xlim()
# plt.hlines(0, xlims[0], xlims[1])

## Decision tree

### Predictive model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=16, min_samples_leaf=1)

clf.fit(X_train, y_train)

preds = clf.predict(X_test)
#print('Test set predictions: {}'.format(preds))
#print('\n')

#print('Train set accuracy: {:.2f}'.format(accuracy_score(y_train, clf.predict(X_train))))
#print('Test set accuracy: {:.2f}'.format(accuracy_score(y_test, clf.predict(X_test))))
print('Train set accuracy: {:.2f}'.format(clf.score(X_train, y_train)))
print('Test set accuracy: {:.2f}'.format(clf.score(X_test, y_test)))
print('\n')

confusion = confusion_matrix(y_test, preds)

print('Confusion matrix:\n', confusion)

# Train models

## k-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

n = 2

clf = KNeighborsClassifier(n)

clf.fit(X_train, y_train)

preds = clf.predict(X_test)
print('Test set predictions: {}'.format(preds))
print('\n')

#print('Train set accuracy: {:.2f}'.format(accuracy_score(y_train, clf.predict(X_train))))
#print('Test set accuracy: {:.2f}'.format(accuracy_score(y_test, clf.predict(X_test))))
print('Train set accuracy: {:.2f}'.format(clf.score(X_train, y_train)))
print('Test set accuracy: {:.2f}'.format(clf.score(X_test, y_test)))
print('\n')

confusion = confusion_matrix(y_test, preds)

print('Confusion matrix:\n', confusion)

Find the best value for n

In [ ]:
training_accuracy = []
test_accuracy = []

n_range = range(1, 11)

for n in n_range:
    clf = KNeighborsClassifier(n)
    clf.fit(X_train, y_train)
    training_accuracy.append(clf.score(X_train, y_train))
    test_accuracy.append(clf.score(X_test, y_test))
    
plt.plot(n_range, training_accuracy, label='training accuracy')
plt.plot(n_range, test_accuracy, label='test accuracy')
plt.xlabel('n neighbors')
plt.ylabel('accuracy')
plt.legend()

## Linear SVM

In [ ]:
from sklearn.svm import SVC

clf = SVC(kernel="linear", C=.1)

clf.fit(X_train, y_train)

preds = clf.predict(X_test)
#print('Test set predictions: {}'.format(preds))
#print('\n')

#print('Train set accuracy: {:.2f}'.format(accuracy_score(y_train, clf.predict(X_train))))
#print('Test set accuracy: {:.2f}'.format(accuracy_score(y_test, clf.predict(X_test))))
print('Train set accuracy: {:.2f}'.format(clf.score(X_train, y_train)))
print('Test set accuracy: {:.2f}'.format(clf.score(X_test, y_test)))
print('\n')

confusion = confusion_matrix(y_test, preds)

print('Confusion matrix:\n', confusion)

# plt.bar(range(X_train.shape[1]), clf.coef_[0], align = 'center', alpha = 0.5)
# plt.xticks(range(X_train.shape[1]), X_train.columns, rotation=90)
# xlims = plt.xlim()
# plt.hlines(0, xlims[0], xlims[1])

Find the best value of C

In [ ]:
training_accuracy = []
test_accuracy = []

C_range = np.logspace(-3, 3, num=7)

for C in C_range:
    clf = SVC(kernel="linear", C=C)
    clf.fit(X_train, y_train)
    training_accuracy.append(clf.score(X_train, y_train))
    test_accuracy.append(clf.score(X_test, y_test))
    
plt.plot(C_range, training_accuracy, label='training accuracy')
plt.plot(C_range, test_accuracy, label='test accuracy')
plt.xlabel('C')
plt.xscale("log")
plt.ylabel('accuracy')
plt.legend()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,preds))
print(classification_report(y_test,preds))

### Split into train, validation, and test set

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=123)

In [ ]:
print(len(y), len(y_train), len(y_val), len(y_test))

In [ ]:
LogReg = LogisticRegression(multi_class='multinomial',max_iter=300)
LogReg.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [ ]:
y_train_pred = cross_val_predict(LogReg, X_train, y_train, cv=3)
confusion_matrix(y_train, y_train_pred)

In [ ]:
precision_score(y_train, y_train_pred, average='micro')

In [ ]:
# # the function uses the list and takes the first of consecutive sequence base on the difference we chose
# def drop_consecutive_row(input_df, delta ):
#     index_list = input_df.index.tolist()
#     index_to_keep = [index_list[0]]
#     for i in range(len(index_list)-1):
#         if index_list[i+1] - index_list[i] > delta:
#             index_to_keep.append(index_list[i+1])
#     return input_df.loc[index_to_keep]